<a href="https://colab.research.google.com/github/astro-petern/docs/blob/main/Engineering_Harness_To_Connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RapidHarness to XML File

Converts rapid harnes files into an xml compatible with the cable checker


In [ ]:
from google.colab import files, drive
import pandas as pd
from ipywidgets import Button, HBox, Output, VBox, Layout, widgets
from IPython.display import display, clear_output
files_to_process = []
import numpy as np


In [ ]:
mapping_file = "/content/AFHN-000226 Manufacturing Data(Wiring Table).csv"
sheed_name = "Wiring Table"
HEADER_IDENTIFIER_VALUE="From"

### File Processing and Confirmation

This code processs a data file - output from Rapid Harness - and make sure it's in the correct format before we use it.

Specifically, it:

*   Sets up a place to show you messages.
*   Has a function that reads a file you give it (either an Excel or CSV file).
*   Tries to clean up the data a bit by removing empty rows and columns.
*   Shows you a preview of the first few rows of the data it read.
*   Asks you to confirm if this is the right file using "Yes" or "No" buttons.
*   If you click "Yes", it saves the data so we can use it later. If you click "No", it skips the file.

In [ ]:
import pandas as pd
from google.colab import files
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output

# Create an output widget to display messages and data
output = Output()
display(output)

# This is where we will store the DataFrame after confirmation
df_confirmed = None
files_to_process = []

def process_and_confirm_file(filepath):
    """Processes a single file and prompts for confirmation."""
    global df_confirmed
    with output:
        clear_output()
        if not filepath:
            print("No file path provided.")
            return

        df = None

        # Check file extension and read accordingly
        if filepath.endswith(('.xlsx', '.xls')):
            print(f"\nAttempting to read Excel file: '{filepath}'")
            try:
                # Read the first sheet as the user indicated a simpler format
                df = pd.read_excel(filepath)

            except Exception as e:
                print(f"Error reading Excel file: {e}")
                return # Skip to the next file on error

        elif filepath.endswith('.csv'):
            print(f"\nAttempting to read CSV file: '{filepath}')")
            df = pd.read_csv(filepath)
        else:
            print("\nError: Unsupported file type. Please upload an Excel (.xlsx, .xls) or CSV (.csv) file.")
            return # Skip to the next file if file type is unsupported

        if df is not None and not df.empty:
            # Remove leading and top NaN padding
            df = df.dropna(axis=0, how='all').dropna(axis=1, how='all')
            df = df.fillna('_')

            print(f"\n--- '{filepath}' Preview ---")
            display(df.head(100))
            print("\nIs this the correct file? Please confirm below.")

            # Create confirmation buttons
            yes_button = Button(description="Yes, correct", button_style='success')
            no_button = Button(description="No, skip", button_style='danger') # Changed text to reflect skipping

            def on_yes_clicked(btn):
                global df_confirmed
                with output:
                    # clear_output()
                    df_confirmed = df
                    print(f"File '{filepath}' confirmed. The DataFrame is now available as 'df_confirmed'.")

            def on_no_clicked(btn):
                with output:
                    clear_output()
                    print(f"File '{filepath}' skipped. Please re-run the file upload cell if you need to process it.")

            yes_button.on_click(on_yes_clicked)
            no_button.on_click(on_no_clicked)

            display(HBox([yes_button, no_button]))
        else:
            print(f"Could not read the file or the file '{filepath}' is empty.")

# Clear the existing files_to_process list and add the new file
files_to_process.clear()
files_to_process.append(mapping_file)


# Iterate through the files_to_process list and process each file
for file_path in files_to_process:
    process_and_confirm_file(file_path)

Output()

### Data Cleaning and Header Identification

This code block focuses on cleaning the confirmed DataFrame (`df_confirmed`) to prepare it for further processing. It specifically addresses the issue of identifying and setting the correct header row, which is embedded within the data rather than being the first row.


In [ ]:
if df_confirmed.empty:
    print("❌ DataFrame is empty after dropping highly sparse rows.")
else:
    df_tmp = df_confirmed.copy()

    # --- DEBUGGED HEADER IDENTIFICATION ---
    # The previous approach (finding the first non-NaN row) incorrectly selected metadata.
    # The correct header starts with the known value 'From'.

    # 1️⃣ Find the index of the row where the first column (index 0) is the header identifier.
    # We must ensure the comparison is robust by checking for string type and stripping whitespace.
    header_row_check = df_tmp.iloc[:, 0].apply(
        lambda x: isinstance(x, str) and x.strip() == HEADER_IDENTIFIER_VALUE
    )

    if header_row_check.any():
        true_header_idx = header_row_check[header_row_check].index[0]
        print(f"✅ True header row containing '{HEADER_IDENTIFIER_VALUE}' detected at original index: {true_header_idx}")

        # 2️⃣ Extract potential header row
        raw_header = df_tmp.loc[true_header_idx].tolist()

        # 3️⃣ Clean header names (This function is kept for general robustness)
        def clean_name(name, idx):
            """Return a safe, non-placeholder header name."""
            if pd.isna(name):
                return f"col_{idx}"
            if not isinstance(name, str):
                return str(name).strip() if name is not None else f"col_{idx}"

            cleaned = name.strip()
            # Treat as placeholder if empty or just underscores/dashes
            if cleaned == "" or cleaned.replace("_", "").replace("-", "") == "":
                return f"col_{idx}"
            return cleaned

        cleaned_header = [clean_name(val, i) for i, val in enumerate(raw_header)]

        print("\nDetected raw header row:", raw_header)
        print("Cleaned header row:", cleaned_header)

        # 4️⃣ Apply new header and drop rows ABOVE and INCLUDING the header row index
        df_cleaned = df_tmp.loc[true_header_idx + 1 :].copy()
        df_cleaned.columns = cleaned_header
        df_cleaned.reset_index(drop=True, inplace=True)

        # 5️⃣ Preview
        print("\n--- Preview of Final Cleaned DataFrame (Wiring Table) ---")
        # Display the first 10 rows for a cleaner view
        display(df_cleaned.head(25))

        print("\n💾 Cleaned DataFrame stored as 'df_confirmed_clean'.")
        # Store the result in the expected output variable
        df_confirmed_clean = df_cleaned

    else:
        print(f"❌ Error: Could not find the expected header row starting with '{HEADER_IDENTIFIER_VALUE}'.")


✅ True header row containing 'From' detected at original index: 8

Detected raw header row: ['From', 'To', 'Conductor', 'Twisted With', 'Size', 'Length', 'From Contact IPN', 'To Contact IPN']
Cleaned header row: ['From', 'To', 'Conductor', 'Twisted With', 'Size', 'Length', 'From Contact IPN', 'To Contact IPN']

--- Preview of Final Cleaned DataFrame (Wiring Table) ---


,From,To,Conductor,Twisted With,Size,Length,From Contact IPN,To Contact IPN
0,J1.1,P15.1,CBL1.White,Blue,26 AWG,0 in,FC_RS422_AB_P,FC_RS422_AB_P
1,J1.2,P15.3,CBL2.White,Blue,26 AWG,0 in,FC_RS422_YZ_P,FC_RS422_YZ_P
2,J1.3,P15.5,W1.White,_,26 AWG,0 in,FORCE_RECOVERY_PIN,FORCE_RECOVERY_PIN
3,J1.4,P15.7,CBL3.White,Blue,26 AWG,0 in,FC_GROUND,FC_GROUND
4,J1.5,P15.9,W2.White,_,26 AWG,0 in,FC_ISO_GSE_GROUND,FC_ISO_GSE_GROUND
5,J1.6,P13.1,W3.White,_,26 AWG,0 in,TDI,TDI
6,J1.7,P13.3,W4.White,_,26 AWG,0 in,TCK_SWCLK,TCK_SWCLK
7,J1.8,P13.5,W5.White,_,26 AWG,0 in,NRST,NRST
8,J1.9,P13.8,CBL4.White,Blue,26 AWG,0 in,ENET_PORT_B_RX_P,ENET_PORT_B_RX_P
9,J1.10,P13.11,CBL5.White,Blue,26 AWG,0 in,ENET_PORT_B_TX_P,ENET_PORT_B_TX_P



💾 Cleaned DataFrame stored as 'df_confirmed_clean'.


XML Generation

Takes a cleaned pandas DataFrame (assumed to be the wiring table) and converts it into an XML string following a specific format.

In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from xml.dom import minidom
from IPython.display import display

# --- Configuration ---
# NOTE: Replace "AFHN-000226 Manufacturing Data(Wiring Table).csv" with your actual file name if it differs.
HEADER_IDENTIFIER_VALUE = "From"

def dataframe_to_xml(df, cable_name="AFHN-000226_HARNESS"):
    """
    Converts the cleaned wiring table DataFrame into the specified XML format.
    Assumes 'From' and 'To' columns contain data in 'Connector.Pin' format (e.g., J1.1).
    """
    if df.empty:
        return "<!-- Empty DataFrame, no XML generated. -->"

    # 1. Prepare XML structure
    root = ET.Element("CableList")
    cable = ET.SubElement(root, "Cable", Name=cable_name)
    connectors_element = ET.SubElement(cable, "Connectors")
    fromto_element = ET.SubElement(cable, "FromTo")

    # 2. Pin and Connector Extraction

    # Create copies of the columns to avoid SettingWithCopyWarning
    df_working = df[['From', 'To']].copy()

    # Separate connector name and pin number using the first '.'
    df_working[['From_Con', 'From_Pin']] = df_working['From'].str.split('.', n=1, expand=True)
    df_working[['To_Con', 'To_Pin']] = df_working['To'].str.split('.', n=1, expand=True)

    # Identify all unique connector names
    unique_connectors = pd.concat([df_working['From_Con'], df_working['To_Con']]).dropna().unique()

    # 3. Generate <Connectors> block
    print("\n3. Generating XML Connectors (assuming 64 pins, as true count is unknown from CSV)...")
    for conn_name in unique_connectors:
        conn = ET.SubElement(connectors_element, "Connector",
                             Name=conn_name,
                             ConName=conn_name,
                             ConID=conn_name)
        # ASSUMPTION: We use '64' for the pin count as a placeholder.
        ET.SubElement(conn, "Pins").text = "64"

    # 4. Generate <FromTo> block (Cx tags)
    print("4. Generating XML Pin Mappings...")
    for _, row in df_working.iterrows():
        # Format: CONNECTOR:PIN (e.g., J1:1)
        from_attr = f"{row['From_Con']}:{row['From_Pin']}"
        to_attr = f"{row['To_Con']}:{row['To_Pin']}"

        # The XML requires a Type attribute, which we infer as "Wire"
        ET.SubElement(fromto_element, "Cx",
                      From=from_attr,
                      To=to_attr,
                      Type="Wire")

    # 5. Format and return XML string
    # Convert ElementTree object to a nicely formatted XML string
    rough_string = ET.tostring(root, 'utf-8')
    reparsed = minidom.parseString(rough_string)

    # Return as string with XML declaration
    return reparsed.toprettyxml(indent="  ")


try:
    # 1. Load the data without a header, as it's embedded later in the file
    df_confirmed = pd.read_csv(FILE_NAME, header=None)

    # Set display options for better visibility
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)

except FileNotFoundError:
    print(f"❌ Error: The file '{FILE_NAME}' was not found. Please ensure it is uploaded.")
    exit()
except Exception as e:
    print(f"❌ An error occurred during file reading: {e}")
    exit()


if df_confirmed.empty:
    print("❌ DataFrame is empty after dropping highly sparse rows.")
else:
    df_tmp = df_confirmed.copy()

    # --- DEBUGGED HEADER IDENTIFICATION ---
    # The correct header starts with the known value 'From'.

    # 1️⃣ Find the index of the row where the first column (index 0) is the header identifier.
    # We must ensure the comparison is robust by checking for string type and stripping whitespace.
    header_row_check = df_tmp.iloc[:, 0].apply(
        lambda x: isinstance(x, str) and x.strip() == HEADER_IDENTIFIER_VALUE
    )

    if header_row_check.any():
        true_header_idx = header_row_check[header_row_check].index[0]
        print(f"✅ True header row containing '{HEADER_IDENTIFIER_VALUE}' detected at original index: {true_header_idx}")

        # 2️⃣ Extract potential header row
        raw_header = df_tmp.loc[true_header_idx].tolist()

        # 3️⃣ Clean header names (This function is kept for general robustness)
        def clean_name(name, idx):
            """Return a safe, non-placeholder header name."""
            if pd.isna(name):
                return f"col_{idx}"
            if not isinstance(name, str):
                return str(name).strip() if name is not None else f"col_{idx}"

            cleaned = name.strip()
            # Treat as placeholder if empty or just underscores/dashes
            if cleaned == "" or cleaned.replace("_", "").replace("-", "") == "":
                return f"col_{idx}"
            return cleaned

        cleaned_header = [clean_name(val, i) for i, val in enumerate(raw_header)]

        # 4️⃣ Apply new header and drop rows ABOVE and INCLUDING the header row index
        df_confirmed_clean = df_tmp.loc[true_header_idx + 1 :].copy()
        df_confirmed_clean.columns = cleaned_header

        # 5️⃣ Preview
        print("\n--- Preview of Final Cleaned DataFrame (Wiring Table) ---")
        print(df_confirmed_clean.head(10).to_markdown(index=False))

        print("\n💾 Cleaned DataFrame stored as 'df_confirmed_clean'.")

        # --- NEW XML GENERATION STEP ---
        xml_output = dataframe_to_xml(df_confirmed_clean, cable_name="AFHN-000226_HARNESS_WIRING")

        # 6️⃣ Print the generated XML
        print("\n--- Generated XML Output ---")
        print(xml_output)

    else:
        print(f"❌ Error: Could not find the expected header row starting with '{HEADER_IDENTIFIER_VALUE}'.")


❌ An error occurred during file reading: name 'FILE_NAME' is not defined
✅ True header row containing 'From' detected at original index: 8

--- Preview of Final Cleaned DataFrame (Wiring Table) ---
| From   | To     | Conductor   | Twisted With   | Size   | Length   | From Contact IPN   | To Contact IPN     |
|:-------|:-------|:------------|:---------------|:-------|:---------|:-------------------|:-------------------|
| J1.1   | P15.1  | CBL1.White  | Blue           | 26 AWG | 0 in     | FC_RS422_AB_P      | FC_RS422_AB_P      |
| J1.2   | P15.3  | CBL2.White  | Blue           | 26 AWG | 0 in     | FC_RS422_YZ_P      | FC_RS422_YZ_P      |
| J1.3   | P15.5  | W1.White    | _              | 26 AWG | 0 in     | FORCE_RECOVERY_PIN | FORCE_RECOVERY_PIN |
| J1.4   | P15.7  | CBL3.White  | Blue           | 26 AWG | 0 in     | FC_GROUND          | FC_GROUND          |
| J1.5   | P15.9  | W2.White    | _              | 26 AWG | 0 in     | FC_ISO_GSE_GROUND  | FC_ISO_GSE_GROUND  |
| J1.6   | 

### Saving XML File

This code block saves the generated XML output to a file.

1.  Define Output Path using `XML_OUTPUT_PATH`.
2.  Print Status
3.  Save XML to File

In [ ]:
XML_OUTPUT_PATH = "/content/AFHN-000226 Manufacturing Data(Wiring Table).xml"
print(f"\n5. Saving XML to file: {XML_OUTPUT_PATH}...")
try:
    # Save the XML to the specified path in Colab
    with open(XML_OUTPUT_PATH, "w") as f:
        f.write(xml_output)
    print(f"✅ XML successfully saved to {XML_OUTPUT_PATH}")
except Exception as e:
    print(f"❌ Error saving XML file: {e}")


5. Saving XML to file: /content/AFHN-000226 Manufacturing Data(Wiring Table).xml...
❌ Error saving XML file: name 'xml_output' is not defined
